In [1]:
import h5py
from torch.autograd import Variable
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.models as models
from tqdm import tqdm

import config
import data
import utils
import resnet as caffe_resnet


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = caffe_resnet.resnet18(pretrained=True)

        def save_output(module, input, output):
            self.buffer = output
        self.model.layer4.register_forward_hook(save_output)

    def forward(self, x):
        self.model(x)
        return self.buffer


def create_vqa_loader(*paths):
    transform = utils.get_transform(config.image_size, config.central_fraction)
    datasets = [data.VSQImages(path, transform=transform) for path in paths]
    dataset = data.Composite(*datasets)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=config.preprocess_batch_size,
        num_workers=config.data_workers,
        shuffle=False,
        pin_memory=True,
    )
    return data_loader


def main():
    cudnn.benchmark = True

    net = Net().cuda()
    net.eval()

    loader = create_vqa_loader(config.train_path, config.val_path)
    features_shape = (
        len(loader.dataset),
        config.output_features,
        config.output_size,
        config.output_size
    )

    with h5py.File(config.preprocessed_path, libver='latest') as fd:
        features = fd.create_dataset('features', shape=features_shape, dtype='float16')
        vsq_ids = fd.create_dataset('ids', shape=(len(loader.dataset),), dtype='int32')

        i = j = 0
        for ids, imgs in tqdm(loader):
            imgs = Variable(imgs).cuda(device=None, non_blocking=True)
            out = net(imgs)

            j = i + imgs.size(0)
            features[i:j, :, :] = out.data.cpu().numpy().astype('float16')
            vsq_ids[i:j] = ids.numpy().astype('int32')
            i = j


if __name__ == '__main__':
    main()


Downloading: "https://s3.amazonaws.com/pytorch/models/resnet18-5c106cde.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/torchvision/transforms/transforms.py:279: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


FileNotFoundError: [Errno 2] No such file or directory: 'train'

will save to logs/2020-12-11_01-14-29.pth



train E000:   0% 0/156 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:69: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

train E000:   0% 0/156 [00:00<?, ?it/s, acc=0.0000, loss=5.8481]
train E000:   1% 1/156 [00:00<02:07,  1.22it/s, acc=0.0000, loss=5.8481]
train E000:   1% 1/156 [00:00<02:07,  1.22it/s, acc=0.2585, loss=5.8478]
train E000:   1% 2/156 [00:00<01:34,  1.64it/s, acc=0.2585, loss=5.8478]
train E000:   1% 2/156 [00:01<01:34,  1.64it/s, acc=0.2814, loss=5.8463]
train E000:   1% 2/156 [00:01<01:34,  1.64it/s, acc=0.3025, loss=5.8460]
train E000:   3% 4/156 [00:01<01:09,  2.20it/s, acc=0.3025, loss=5.8460]
train E000:   3% 4/156 [00:01<01:09,  2.20it/s, acc=0.3063, loss=5.8454]
train E000:   3% 4/156 [00:01<01:09,  2.20it/s, acc=0.3466, loss=5.8488]
train E000:   4% 6/156 [00:01<00:51,  2.92it/s, acc=0.3466, loss=5.8488]
train E000:

train E000:  88% 138/156 [00:11<00:01, 13.06it/s, acc=0.2953, loss=4.4274]
train E000:  88% 138/156 [00:11<00:01, 13.06it/s, acc=0.2681, loss=4.4244]
train E000:  90% 140/156 [00:11<00:01, 12.85it/s, acc=0.2681, loss=4.4244]
train E000:  90% 140/156 [00:11<00:01, 12.85it/s, acc=0.3197, loss=4.4170]
train E000:  90% 140/156 [00:11<00:01, 12.85it/s, acc=0.3095, loss=4.4134]
train E000:  91% 142/156 [00:11<00:01, 13.46it/s, acc=0.3095, loss=4.4134]
train E000:  91% 142/156 [00:11<00:01, 13.46it/s, acc=0.2837, loss=4.4091]
train E000:  91% 142/156 [00:11<00:01, 13.46it/s, acc=0.2908, loss=4.4048]
train E000:  92% 144/156 [00:11<00:00, 13.95it/s, acc=0.2908, loss=4.4048]
train E000:  92% 144/156 [00:11<00:00, 13.95it/s, acc=0.2889, loss=4.4002]
train E000:  92% 144/156 [00:11<00:00, 13.95it/s, acc=0.3144, loss=4.3957]
train E000:  94% 146/156 [00:11<00:00, 13.91it/s, acc=0.3144, loss=4.3957]
train E000:  94% 146/156 [00:11<00:00, 13.91it/s, acc=0.3370, loss=4.3903]
train E000:  94% 146/156 

train E001:  60% 93/156 [00:07<00:04, 13.40it/s, acc=0.3318, loss=3.6742]
train E001:  60% 93/156 [00:07<00:04, 13.40it/s, acc=0.3417, loss=3.6739]
train E001:  61% 95/156 [00:07<00:04, 13.53it/s, acc=0.3417, loss=3.6739]
train E001:  61% 95/156 [00:07<00:04, 13.53it/s, acc=0.3799, loss=3.6687]
train E001:  61% 95/156 [00:07<00:04, 13.53it/s, acc=0.3350, loss=3.6682]
train E001:  62% 97/156 [00:07<00:04, 13.74it/s, acc=0.3350, loss=3.6682]
train E001:  62% 97/156 [00:07<00:04, 13.74it/s, acc=0.3548, loss=3.6687]
train E001:  62% 97/156 [00:08<00:04, 13.74it/s, acc=0.3106, loss=3.6713]
train E001:  63% 99/156 [00:08<00:04, 13.59it/s, acc=0.3106, loss=3.6713]
train E001:  63% 99/156 [00:08<00:04, 13.59it/s, acc=0.3251, loss=3.6698]
train E001:  63% 99/156 [00:08<00:04, 13.59it/s, acc=0.3220, loss=3.6673]
train E001:  65% 101/156 [00:08<00:04, 13.32it/s, acc=0.3220, loss=3.6673]
train E001:  65% 101/156 [00:08<00:04, 13.32it/s, acc=0.3065, loss=3.6696]
train E001:  65% 101/156 [00:08<00:0

train E002:  31% 49/156 [00:04<00:08, 13.29it/s, acc=0.3367, loss=3.3832]
train E002:  31% 49/156 [00:04<00:08, 13.29it/s, acc=0.3292, loss=3.3842]
train E002:  31% 49/156 [00:04<00:08, 13.29it/s, acc=0.3049, loss=3.3858]
train E002:  33% 51/156 [00:04<00:08, 12.57it/s, acc=0.3049, loss=3.3858]
train E002:  33% 51/156 [00:04<00:08, 12.57it/s, acc=0.3342, loss=3.3880]
train E002:  33% 51/156 [00:04<00:08, 12.57it/s, acc=0.3063, loss=3.3887]
train E002:  34% 53/156 [00:04<00:08, 12.64it/s, acc=0.3063, loss=3.3887]
train E002:  34% 53/156 [00:04<00:08, 12.64it/s, acc=0.2804, loss=3.3921]
train E002:  34% 53/156 [00:04<00:08, 12.64it/s, acc=0.3533, loss=3.3911]
train E002:  35% 55/156 [00:04<00:07, 13.06it/s, acc=0.3533, loss=3.3911]
train E002:  35% 55/156 [00:04<00:07, 13.06it/s, acc=0.2946, loss=3.3929]
train E002:  35% 55/156 [00:05<00:07, 13.06it/s, acc=0.2852, loss=3.3932]
train E002:  37% 57/156 [00:05<00:07, 13.31it/s, acc=0.2852, loss=3.3932]
train E002:  37% 57/156 [00:05<00:07, 

...

train E009:   7% 11/156 [00:01<00:22,  6.52it/s, acc=0.4080, loss=2.6471]
train E009:   8% 13/156 [00:01<00:18,  7.62it/s, acc=0.4080, loss=2.6471]
train E009:   8% 13/156 [00:01<00:18,  7.62it/s, acc=0.3704, loss=2.6471]
train E009:   8% 13/156 [00:01<00:18,  7.62it/s, acc=0.3367, loss=2.6503]
train E009:  10% 15/156 [00:01<00:16,  8.69it/s, acc=0.3367, loss=2.6503]
train E009:  10% 15/156 [00:01<00:16,  8.69it/s, acc=0.3467, loss=2.6518]
train E009:  10% 15/156 [00:02<00:16,  8.69it/s, acc=0.3029, loss=2.6553]
train E009:  11% 17/156 [00:02<00:14,  9.47it/s, acc=0.3029, loss=2.6553]
train E009:  11% 17/156 [00:02<00:14,  9.47it/s, acc=0.3436, loss=2.6571]
train E009:  11% 17/156 [00:02<00:14,  9.47it/s, acc=0.3758, loss=2.6568]
train E009:  12% 19/156 [00:02<00:13, 10.28it/s, acc=0.3758, loss=2.6568]
train E009:  12% 19/156 [00:02<00:13, 10.28it/s, acc=0.3545, loss=2.6576]
train E009:  12% 19/156 [00:02<00:13, 10.28it/s, acc=0.3537, loss=2.6610]
train E009:  13% 21/156 [00:02<00:12, 

train E009:  98% 153/156 [00:12<00:00, 14.14it/s, acc=0.3065, loss=2.8159]
train E009:  98% 153/156 [00:12<00:00, 14.14it/s, acc=0.3652, loss=2.8163]
train E009:  98% 153/156 [00:12<00:00, 14.14it/s, acc=0.3346, loss=2.8155]
train E009:  99% 155/156 [00:12<00:00, 14.17it/s, acc=0.3346, loss=2.8155]
train E009: 100% 156/156 [00:12<00:00, 12.45it/s, acc=0.3728, loss=2.8141]

val E009:   0% 0/34 [00:00<?, ?it/s]
val E009:   0% 0/34 [00:00<?, ?it/s, acc=0.2477, loss=2.4220]
val E009:   3% 1/34 [00:00<00:18,  1.76it/s, acc=0.2477, loss=2.4220]
val E009:   3% 1/34 [00:00<00:18,  1.76it/s, acc=0.2457, loss=2.2918]
val E009:   3% 1/34 [00:00<00:18,  1.76it/s, acc=0.2313, loss=2.2842]
val E009:   9% 3/34 [00:00<00:12,  2.39it/s, acc=0.2313, loss=2.2842]
val E009:   9% 3/34 [00:00<00:12,  2.39it/s, acc=0.2346, loss=2.2108]
val E009:   9% 3/34 [00:00<00:12,  2.39it/s, acc=0.2295, loss=2.2607]
val E009:  15% 5/34 [00:00<00:09,  3.19it/s, acc=0.2295, loss=2.2607]
val E009:  15% 5/34 [00:00<00:09,  

train E010:  69% 108/156 [00:08<00:03, 13.99it/s, acc=0.3407, loss=2.8190]
train E010:  69% 108/156 [00:08<00:03, 13.99it/s, acc=0.3607, loss=2.8187]
train E010:  69% 108/156 [00:08<00:03, 13.99it/s, acc=0.3561, loss=2.8200]
train E010:  71% 110/156 [00:08<00:03, 13.91it/s, acc=0.3561, loss=2.8200]
train E010:  71% 110/156 [00:08<00:03, 13.91it/s, acc=0.3587, loss=2.8214]
train E010:  71% 110/156 [00:09<00:03, 13.91it/s, acc=0.4188, loss=2.8205]
train E010:  72% 112/156 [00:09<00:03, 13.48it/s, acc=0.4188, loss=2.8205]
train E010:  72% 112/156 [00:09<00:03, 13.48it/s, acc=0.3826, loss=2.8212]
train E010:  72% 112/156 [00:09<00:03, 13.48it/s, acc=0.2975, loss=2.8236]
train E010:  73% 114/156 [00:09<00:03, 13.26it/s, acc=0.2975, loss=2.8236]
train E010:  73% 114/156 [00:09<00:03, 13.26it/s, acc=0.3642, loss=2.8252]
train E010:  73% 114/156 [00:09<00:03, 13.26it/s, acc=0.3353, loss=2.8253]
train E010:  74% 116/156 [00:09<00:03, 13.10it/s, acc=0.3353, loss=2.8253]
train E010:  74% 116/156 

train E011:  40% 62/156 [00:05<00:06, 13.45it/s, acc=0.3423, loss=2.6270]
train E011:  40% 62/156 [00:05<00:06, 13.45it/s, acc=0.3949, loss=2.6286]
train E011:  41% 64/156 [00:05<00:06, 13.66it/s, acc=0.3949, loss=2.6286]
train E011:  41% 64/156 [00:05<00:06, 13.66it/s, acc=0.3657, loss=2.6316]
train E011:  41% 64/156 [00:05<00:06, 13.66it/s, acc=0.3597, loss=2.6332]
train E011:  42% 66/156 [00:05<00:06, 13.31it/s, acc=0.3597, loss=2.6332]
train E011:  42% 66/156 [00:05<00:06, 13.31it/s, acc=0.3692, loss=2.6340]
train E011:  42% 66/156 [00:05<00:06, 13.31it/s, acc=0.3186, loss=2.6346]
train E011:  44% 68/156 [00:05<00:06, 13.19it/s, acc=0.3186, loss=2.6346]
train E011:  44% 68/156 [00:05<00:06, 13.19it/s, acc=0.3264, loss=2.6360]
train E011:  44% 68/156 [00:05<00:06, 13.19it/s, acc=0.3418, loss=2.6358]
train E011:  45% 70/156 [00:05<00:06, 13.51it/s, acc=0.3418, loss=2.6358]
train E011:  45% 70/156 [00:06<00:06, 13.51it/s, acc=0.3370, loss=2.6374]
train E011:  45% 70/156 [00:06<00:06, 

train E012:  10% 16/156 [00:01<00:14,  9.44it/s, acc=0.3333, loss=2.7189]
train E012:  10% 16/156 [00:01<00:14,  9.44it/s, acc=0.3703, loss=2.7158]
train E012:  12% 18/156 [00:01<00:13, 10.40it/s, acc=0.3703, loss=2.7158]
train E012:  12% 18/156 [00:02<00:13, 10.40it/s, acc=0.3689, loss=2.7161]
train E012:  12% 18/156 [00:02<00:13, 10.40it/s, acc=0.3304, loss=2.7166]
train E012:  13% 20/156 [00:02<00:12, 11.22it/s, acc=0.3304, loss=2.7166]
train E012:  13% 20/156 [00:02<00:12, 11.22it/s, acc=0.3373, loss=2.7167]
train E012:  13% 20/156 [00:02<00:12, 11.22it/s, acc=0.3424, loss=2.7164]
train E012:  14% 22/156 [00:02<00:10, 12.18it/s, acc=0.3424, loss=2.7164]
train E012:  14% 22/156 [00:02<00:10, 12.18it/s, acc=0.3731, loss=2.7154]
train E012:  14% 22/156 [00:02<00:10, 12.18it/s, acc=0.3299, loss=2.7149]
train E012:  15% 24/156 [00:02<00:10, 12.27it/s, acc=0.3299, loss=2.7149]
train E012:  15% 24/156 [00:02<00:10, 12.27it/s, acc=0.3740, loss=2.7121]
train E012:  15% 24/156 [00:02<00:10, 